In [1]:
import pandas as pd
import unicodedata

# Funçoes

In [2]:
def slugify(s: str) -> str:
  s = s.strip().lower()
  s = unicodedata.normalize("NFD", s)
  s = s.encode("ascii", "ignore")
  s = s.decode("utf-8")
  s = s.lower().strip()
  return s

# Tratamento

In [ ]:
columns_order_rename = {'NM_DIST': 'distrito',
  '0 a 4 anos': '0 a 4 anos - Censo 2022',
  '5 a 9 anos': '5 a 9 anos - Censo 2022',
  '10 a 14 anos': '10 a 14 anos - Censo 2022',
  '15 a 19 anos': '15 a 19 anos - Censo 2022',
  '5 a 19 anos': '5 a 19 anos - Censo 2022',
  'quantidade_matricula_infantil_creche': 'quantidade_matricula_infantil_creche - INEP 2023',
  'quantidade_matricula_educacao_basica': 'quantidade_matricula_educacao_basica - INEP 2023',
  'quantidade_matricula_infantil': 'quantidade_matricula_infantil - INEP 2023',
  'quantidade_matricula_infantil_pre_escola': 'quantidade_matricula_infantil_pre_escola - INEP 2023',
  'quantidade_matricula_fundamental': 'quantidade_matricula_fundamental - INEP 2023',
  'quantidade_matricula_medio': 'quantidade_matricula_medio - INEP 2023',
  'quantidade_matricula_idade_0_3': 'quantidade_matricula_idade_0_3 - INEP 2023',
  'quantidade_matricula_idade_4_5': 'quantidade_matricula_idade_4_5 - INEP 2023',
  'quantidade_matricula_idade_6_10': 'quantidade_matricula_idade_6_10 - INEP 2023',
  'quantidade_matricula_idade_11_14': 'quantidade_matricula_idade_11_14 - INEP 2023',
  'quantidade_matricula_idade_15_17': 'quantidade_matricula_idade_15_17 - INEP 2023',
  'quantidade_matricula_idade_18': 'quantidade_matricula_idade_18 - INEP 2023',
  'taxa de universalização da educação infantil': 'taxa de universalização da educação infantil - 2023',
  'taxa de universalização da educação básica': 'taxa de universalização da educação básica - 2023'
  }

columns_to_sum = list(columns_order_rename.keys())[6:-2]

# Load Tabelas

df_escolas = pd.read_parquet('/content/escolas_distritos_geometry.parquet')
df_censo_setor = pd.read_parquet('/content/ibge_censo_setor_censitario.parquet')

# Preparar Tabela para os indicadores

df_indicadores = df_escolas.groupby('nm_distrito_municipal')[columns_to_sum].sum()
df_indicadores.index = [slugify(index) for index in df_indicadores.index]

# Diminuir o tamanho tabela Censo e preparar para juntar com tabela Indicadores

df_censo_setor_to_indicadores = df_censo_setor[['NM_DIST', '0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos']]
df_censo_setor_to_indicadores['5 a 19 anos'] = df_censo_setor_to_indicadores[['5 a 9 anos', '10 a 14 anos', '15 a 19 anos']].astype(int).sum(axis=1)

df_censo_setor_to_indicadores.index = df_censo_setor_to_indicadores.NM_DIST.apply(slugify)

# Juntar tabelas

df_indicadores = pd.concat([df_indicadores, df_censo_setor_to_indicadores], axis=1)

# Criar Indicadores

df_indicadores['taxa de universalização da educação infantil'] = ((df_indicadores['quantidade_matricula_infantil_creche'].astype(int) /  df_indicadores['0 a 4 anos'].astype(int)) * 100).round(2)
df_indicadores['taxa de universalização da educação básica'] = ((df_indicadores['quantidade_matricula_educacao_basica'].astype(int) /  df_indicadores['5 a 19 anos'].astype(int)) * 100).round(2)

# Organizar e mudar nome das colunas

df_indicadores.rename(columns=columns_order_rename, inplace=True)
df_indicadores = df_indicadores.loc[:, columns_order_rename.values()]

# Salve

df_indicadores.to_excel("indicadores_educação.xlsx", index=False)